## 讀取視訊鏡頭

In [1]:
import cv2

cap = cv2.VideoCapture(0) # 抓取視訊鏡頭

# 設定大小，預設為640*480，不可設定再大，可以設定再小
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 320)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 240)

width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
print("Image Size: %d x %d" % (width, height))
while(True):
    # 從攝影機擷取一張影像
    ret, frame = cap.read()
    
    # 顯示圖片
    cv2.imshow('frame', frame)

    # 若按下 q 鍵則離開迴圈
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
# 釋放攝影機
cap.release()

# 關閉所有 OpenCV 視窗
cv2.destroyAllWindows()

Image Size: 320 x 240


## 讀取影片

In [5]:
import cv2

cap = cv2.VideoCapture('./video2.mp4') # 預設為640*480，不可設定再大，可以設定再小

while(True):
    # 從攝影機擷取一張影像
    ret, frame = cap.read()
    
    if ret == False:
        break
    # 顯示圖片
    cv2.imshow('frame', frame)

    # 若按下 q 鍵則離開迴圈
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
# 釋放攝影機
cap.release()

# 關閉所有 OpenCV 視窗
cv2.destroyAllWindows()

In [20]:
import cv2
import numpy as np

# 開啟網路攝影機
cap = cv2.VideoCapture('./video2.mp4')

# 設定影像尺寸
width = 1280
height = 960

# 設定擷取影像的尺寸大小
cap.set(cv2.CAP_PROP_FRAME_WIDTH, width)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, height)

# 計算畫面面積
area = width * height

# 初始化平均影像
ret, frame = cap.read()
avg = cv2.blur(frame, (4, 4))
avg_float = np.float32(avg)
i = 1
j = 1
while(cap.isOpened()):
    # 讀取一幅影格
    ret, frame = cap.read()

    # 若讀取至影片結尾，則跳出
    if ret == False:
        break

    # 邊緣模糊處理 (平均濾波)
    blur = cv2.blur(frame, (4, 4))

    # 上一幀與這一幀的差異 (可抓到移動中物品)
    diff = cv2.absdiff(avg, blur)

    # 將圖片轉為灰階
    gray = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)

    # 篩選出變動程度大於門檻值的區域 (要先變灰階)
    # "<25":灰階變為 0, ">25":灰階變為 1
    ret, thresh = cv2.threshold(gray, 30, 255, cv2.THRESH_BINARY)

    # 使用型態轉換函數去除雜訊
    kernel = np.ones((5, 5), np.uint8) # 濾波器
    thresh = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=2) # 去白噪點
    thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel, iterations=2) # 去黑噪點

    # 輪廓檢測 (要先二值化變黑白)，產生等高線
    contours , cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for c in contours:
        # 忽略太小的區域
        if cv2.contourArea(c) < 2500:
            continue
        i += 1
        # 偵測到物體，可以自己加上處理的程式碼在這裡...

        # 計算等高線的外框範圍
        (x, y, w, h) = cv2.boundingRect(c)
#         if i%300 == 0:
#             cv2.imwrite('./move/'+str(j)+'_output.png', frame[x:x+w,y:y+h])
#             j += 1
        # 畫出外框
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # 畫出等高線（除錯用）
    cv2.drawContours(frame, contours, -1, (0, 255, 255), 2)

    # 顯示偵測結果影像
    cv2.imshow('frame', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    # 更新平均影像
    cv2.accumulateWeighted(blur, avg_float, 0.01)
    avg = cv2.convertScaleAbs(avg_float)

cap.release()
cv2.destroyAllWindows()

## absdiff
https://makerpro.cc/2018/11/opencv-background-subtractor/

In [22]:
import cv2
import numpy as np

cap = cv2.VideoCapture('./video2.mp4')
i = 1
j = 1
while(True):
    t0 = cap.read()[1]
    t1 = cap.read()[1]
    
    grey1 = cv2.cvtColor(t0, cv2.COLOR_BGR2GRAY)
    blur1 = cv2.GaussianBlur(grey1,(7,7),0)
    
    grey2 = cv2.cvtColor(t1, cv2.COLOR_BGR2GRAY)
    blur2 = cv2.GaussianBlur(grey2,(7,7),0)

    d = cv2.absdiff(blur1, blur2)
    
    (T, img) = cv2.threshold(d, 30, 255, cv2.THRESH_BINARY)
    
    img = cv2.dilate(img, None, iterations=36)

    img = cv2.erode(img, None, iterations=36)

    img = cv2.dilate(img, None, iterations=12)
    contours, hierarchy = cv2.findContours(img,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    for c in contours:
        if cv2.contourArea(c) < 3000:
            continue
        (x, y, w, h) = cv2.boundingRect(c)
#         cv2.drawContours(t1, contours, -1, (0,0,255), 2)
        cv2.rectangle(t1,(x,y),(x+w,y+h), (0,255,0),2)
        if i % 20 == 0:
            cv2.imwrite('./move/'+str(j)+'_output.png', t1[y:y+h,x:x+w])
            j += 1
        i += 1
    
    cv2.imshow('frame', t1)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

In [1]:
import numpy as np
import cv2
import os

#----- Your configuration ---------------------------------------------------------------

# 如果您使用HDMI螢幕, 請將displayDevice設為1, 若像本文一樣使用TFT LCD接到RPI, 請設為2
displayDevice = 2  # 1--> LCD monitor  2--> ILI9341 TFT

#最終畫面要顯示的種類 dislpayType, 物體外形或影像, 您可以分別都試看看.
dislpayType = 2  #1--> Contour  2--> Image

#抓取到的移動物體的標示方法
markType = 3  #1--> Draw edge  2-->Box selection  3--> Draw & Box


#--Functions------------------------------------------------------------------------------

def wait():
    raw_input('Press Enter')

def createFolder(pathFolder):
    if(not os.path.exists(pathFolder)):
        os.makedirs(pathFolder)

def writeImage(num, img):
    global imgFolder
    imgFile = ("G{}.png".format(num))
    cv2.imwrite(imgFolder + imgFile, img)

#-----------------------------------------------------------------------------------------

cap = cv2.VideoCapture(0)

t0 = cap.read()[1]
t1 = cap.read()[1]

zeros = np.zeros(t0.shape[:2], dtype = "uint8")

i = 0
while(True):
    grey1 = cv2.cvtColor(t0, cv2.COLOR_BGR2GRAY)
    grey2 = cv2.cvtColor(t1, cv2.COLOR_BGR2GRAY)

    blur1 = cv2.GaussianBlur(grey1,(7,7),0)
    blur2 = cv2.GaussianBlur(grey2,(5,5),0)

    d = cv2.absdiff(blur1, blur2)

    ret, th = cv2.threshold( d, 10, 255, cv2.THRESH_BINARY )

    dilated=cv2.dilate(th, None, iterations=1)


    contours, hierarchy = cv2.findContours(dilated,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    areas = [cv2.contourArea(c) for c in contours]

    if(dislpayType==1):
        layer = np.zeros(t0.shape[:2], dtype = "uint8")
        markColor = 255
    elif(dislpayType==2):
        layer = t1
        markColor = (0,255,0)

    if(len(areas)>0):
        max_index = np.argmax(areas)
        cnt=contours[max_index]
        x,y,w,h = cv2.boundingRect(cnt)
        if(areas[max_index]>5000):
            if(markType==1 or markType==3):
                cv2.drawContours(layer, cnt, -1, markColor, 2)
            if(markType==2 or markType==3):
                cv2.rectangle(layer,(x,y),(x+w,y+h), markColor,2)

    layer2 = np.vstack((cv2.merge([zeros, d, zeros]), cv2.merge([zeros, zeros, th]), layer ))
    cv2.imshow('frame', layer2)

    t0=t1
    t1=cap.read()[1]    

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    i = i + 1

cap.release()

## MOG2
https://makerpro.cc/2018/11/opencv-background-subtractor/ <br>
https://codertw.com/程式語言/558529/

In [10]:
import numpy as np
import cv2

cap = cv2.VideoCapture(0)
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(3,3))
mog = cv2.createBackgroundSubtractorMOG2()

while True:
    t0 = cap.read()[1]
    mogmask = mog.apply(t0)
    
    mogmask = cv2.morphologyEx(mogmask, cv2.MORPH_OPEN, kernel)
    contours, hierarchy = cv2.findContours(mogmask,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    for c in contours:
        perimeter = cv2.arcLength(c,True)
        if perimeter > 188:
            x,y,w,h = cv2.boundingRect(c)
            cv2.rectangle(t0,(x,y),(x+w,y+h),(0,255,0),2)    
            cv2.imshow('frame',t0)
            cv2.imshow('fgmask', mogmask)
          
    if cv2.waitKey(30) & 0xff == 27: # 27 為 Esc
        break
cap.release()
cv2.destroyAllWindows()

In [6]:
chr(97)

'a'